## Loading library

In [1]:
library(edgeR)
library(ggplot2)
library("ggrepel")

Loading required package: limma

Warning message:
"replacing previous import 'vctrs::data_frame' by 'tibble::data_frame' when loading 'dplyr'"


## Read the data
you should have to prepare two data:
1. FPKM or count matrix
2. metadata (with the same order of data set 1)
3. gene list which you can convert gene id to gene name

one you have all your data, you need to subset you data base on the comparison you want to do.

In [5]:
#read table (RNA_seq)
datExpr = read.csv("countMatrix.csv", row.name=1, sep=",", header = T, check.names = FALSE)
head(datExpr)

,12mo_hipp_F_5xFADHEMI_11932_rsem.genes.results,12mo_hipp_M_5xFADHEMI:SPI1HO_12188_rsem.genes.results,4mo_hipp_F_5xFADWT:SPI1HO_13262_rsem.genes.results,12mo_hipp_F_5xFADWT_11643_rsem.genes.results,12mo_hipp_F_5xFADWT:SPI1HO_12194_rsem.genes.results,12mo_hipp_F_5xFADWT_11412_rsem.genes.results,4mo_hipp_M_5xFADHEMI:SPI1HO_13251_rsem.genes.results,12mo_hipp_F_5xFADHEMI:SPI1HO_12204_rsem.genes.results,4mo_hipp_M_5xFADHEMI_11631_rsem.genes.results,4mo_hipp_F_5xFADWT:SPI1HO_13267_rsem.genes.results,...,12mo_hipp_M_5xFADWT:SPI1HO_12180_rsem.genes.results,12mo_hipp_F_5xFADHEMI:SPI1HO_12203_rsem.genes.results,12mo_hipp_F_5xFADWT_11414_rsem.genes.results,4mo_hipp_M_5xFADWT_11620_rsem.genes.results,4mo_hipp_F_5xFADWT_11628_rsem.genes.results,12mo_hipp_F_5xFADWT:SPI1HO_12205_rsem.genes.results,12mo_hipp_M_5xFADWT_11650_rsem.genes.results,12mo_hipp_F_5xFADHEMI_11644_rsem.genes.results,12mo_hipp_F_5xFADWT_11656_rsem.genes.results,4mo_hipp_M_5xFADHEMI:SPI1HO_13268_rsem.genes.results
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000000001.4,1009,1011,745,760,752,808,712,912,782,734,...,654,745,532,675,791,701,797,663,766,920
ENSMUSG00000000003.15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000000028.15,62,39,41,36,25,10,23,52,24,30,...,13,40,22,18,22,30,27,34,20,29
ENSMUSG00000000031.16,21,6,12,3,7,0,5,10,6,10,...,1,10,6,2,8,20,1,8,2,5
ENSMUSG00000000037.16,5,11,27,18,14,19,14,30,10,9,...,13,18,19,12,6,19,20,19,12,13
ENSMUSG00000000049.11,4,6,8,1,5,4,5,3,5,31,...,0,10,0,7,12,4,8,2,8,17


For each comparison, you need to repeat this part

## Subset data

if you want to only consider subset of samples instead of whole you should remove those samples from count matrix
In this part of the code, you will keep the information about those samples that you care about it!

In [2]:
datTraits = read.csv("experimentList.csv", row.name=1, sep=",", header = T)
datTraits$keep = FALSE
head(datTraits)

,Time.point,Tissue,Sex,Group,keep
,<chr>,<chr>,<chr>,<chr>,<lgl>
12mo_hipp_F_5xFADHEMI_11932_rsem.genes.results,12mo,hipp,F,5xFADHEMI,FALSE
12mo_hipp_M_5xFADHEMI:SPI1HO_12188_rsem.genes.results,12mo,hipp,M,5xFADHEMI:SPI1HO,FALSE
4mo_hipp_F_5xFADWT:SPI1HO_13262_rsem.genes.results,4mo,hipp,F,5xFADWT:SPI1HO,FALSE
12mo_hipp_F_5xFADWT_11643_rsem.genes.results,12mo,hipp,F,5xFADWT,FALSE
12mo_hipp_F_5xFADWT:SPI1HO_12194_rsem.genes.results,12mo,hipp,F,5xFADWT:SPI1HO,FALSE
12mo_hipp_F_5xFADWT_11412_rsem.genes.results,12mo,hipp,F,5xFADWT,FALSE


In [3]:
## you should cahnge column name and keep variable based on your comparison you want to do
## if you need to do filtering in more than one columns you should copy and pase this part of the code
column_name = 'Time.point'
keep = c("4mo")
for (i in c(1:dim(datTraits)[1])){
  if (datTraits[i, column_name] %in% keep){
      datTraits[i, "keep"] = TRUE
  }
}
datTraits_selected = datTraits[datTraits$keep, ]
column_name = 'Group'
keep = c("5xFADHEMI:SPI1HO", "5xFADWT:SPI1HO")
for (i in c(1:dim(datTraits_selected)[1])){
  if (datTraits_selected[i, column_name] %in% keep){
      datTraits_selected[i, "keep"] = TRUE
  }
}

In [6]:
keep = datTraits_selected[datTraits_selected$keep, ]
datExpr_selected = datExpr[, rownames(keep)]

## Define group

after you get your data it's time to define the group which is bassically categorize the parameter you want to investigate by doing DEG
here you need to define which parameter you want to investigte

In [7]:
parameter = "Group"
options = unique(keep[,parameter])
groups = rep(0, dim(datExpr_selected)[2])
for (i in c(1:dim(datExpr_selected)[2])) {
  if (keep[i,parameter] == options[1]){
    groups[i] = 1
  }
  if (keep[i,parameter] == options[2]){
    groups[i] = 2
  }
}

## Running DEG using edgR

Now we have everythings, we only need to run it.

In [8]:
y = DGEList(counts = datExpr_selected, group=groups)
y = calcNormFactors(y)
design = model.matrix(~groups)
y = estimateDisp(y, design)

keep = filterByExpr(y)
y = y[keep, , keep.lib.sizes=FALSE]

#Testing for DE Genes
et = exactTest(y)

#extract table from the exact test( here is where we know if they are DE or not)
et_out = (topTags(et, n=Inf, adjust.method = "BH"))
et = et_out$table

In [9]:
# label DE genes
# you can change therosholds base on your data!
et$DE = ""
et$DE[et$FDR < 0.05 & et$logFC > 0 ] = "Up"
et$DE[et$FDR < 0.05 & et$logFC < 0 ] = "Down"
et$DE[ et$DE == "" ] = "No"

table(et$DE)


 Down    No    Up 
    1 16281     2 

## convert gene id to gene name
if you want to have gene name as well, you should run below chunk of code and if not just skip this part.

In [14]:
et$gene_name = rownames(et)

#read gene list to convert gene ID to gene list
annot = read.table("genelist.csv", sep=",", header = T)

## convert gene ID to gene Name
for (i in c(1:dim(et)[1])){
  et$gene_name[i] = annot$gene_name[which(et$gene_name[i] == annot$gene_id)]
}

In [46]:
# save table
write.table(et, "DEG/table_DEG_4mon_5xFADHEMI:SPI1HO_SPI1HO.tsv", sep="\t", row.names = F, quote = F)

## volcano plot

There is only one step left! plotting the results
if you change the default threshold you need to change it here as well!
For doing that `geom_vline` and `geom_hline` function

In [47]:
et$label = et$gene_name
for (i in c(1:dim(et)[1])) {
  if (et$DE[i] == "No") {
    et$label[i] = ""
  }
}

pdf(paste0("DEG/plot_DEG_4mon_5xFADHEMI:SPI1HO_SPI1HO.pdf"), width = 6, height = 6)
ggplot(data=et, aes(x=logFC, y=-log10(FDR), col=DE, label=label)) +
  geom_point() + 
  theme_minimal() +
  geom_text_repel() +
  scale_color_manual(values=c("blue", "black", "red")) +
  geom_vline(xintercept=c(-0, 0), col="red") +
  geom_hline(yintercept=-log10(0.05), col="red")
dev.off();
dev.off();

Warning message:
"ggrepel: 1708 unlabeled data points (too many overlaps). Consider increasing max.overlaps"


png 
  2

null device 
          1